# Variance of one stock


In [5]:
import sys
!{sys.executable} -m pip install --upgrade pip 
!{sys.executable} -m pip install --upgrade wheel setuptools build 
!{sys.executable} -m pip install -r requirements.txt

In [6]:
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

ImportError: cannot import name 'ABCIndexClass'

In [7]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

NameError: name 'plt' is not defined

### data bundle

In [ ]:
import os
import quiz_helper
from zipline.data import bundles

In [ ]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

### Build pipeline engine

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

### View Data¶
With the pipeline engine built, let's get the stocks at the end of the period in the universe we're using. We'll use these tickers to generate the returns data for the our risk model.

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

In [ ]:
len(universe_tickers)

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## Get pricing data helper function

In [ ]:
from quiz_helper import get_pricing

## get pricing data into a dataframe

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## Let's look at a two stock portfolio

Let's pretend we have a portfolio of two stocks.  We'll pick Apple and Microsoft in this example.

In [ ]:
aapl_col = returns_df.columns[3]
msft_col = returns_df.columns[312]
asset_return_1 = returns_df[aapl_col].rename('asset_return_aapl')
asset_return_2 = returns_df[msft_col].rename('asset_return_msft')
asset_return_df = pd.concat([asset_return_1,asset_return_2],axis=1)
asset_return_df.head(2)

## Factor returns
Let's make up a "factor" by taking an average of all stocks in our list.  You can think of this as an equal weighted index of the 490 stocks, kind of like a measure of the "market".  We'll also make another factor by calculating the median of all the stocks.  These are mainly intended to help us generate some data to work with.  We'll go into how some common risk factors are generated later in the lessons.

Also note that we're setting axis=1 so that we calculate a value for each time period (row) instead of one value for each column (assets).

In [ ]:
factor_return_1 = returns_df.mean(axis=1)
factor_return_2 = returns_df.median(axis=1)
factor_return_l = [factor_return_1, factor_return_2]

## Factor exposures

Factor exposures refer to how "exposed" a stock is to each factor.  We'll get into this more later.  For now, just think of this as one number for each stock, for each of the factors.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
"""
For now, just assume that we're calculating a number for each 
stock, for each factor, which represents how "exposed" each stock is
to each factor. 
We'll discuss how factor exposure is calculated later in the lessons.
"""
def get_factor_exposures(factor_return_l, asset_return):
    lr = LinearRegression()
    X = np.array(factor_return_l).T
    y = np.array(asset_return.values)
    lr.fit(X,y)
    return lr.coef_

In [ ]:
factor_exposure_l = []
for i in range(len(asset_return_df.columns)):
    factor_exposure_l.append(
        get_factor_exposures(factor_return_l,
                             asset_return_df[asset_return_df.columns[i]]
                            ))
    
factor_exposure_a = np.array(factor_exposure_l)

In [ ]:
print(f"factor_exposures for asset 1 {factor_exposure_a[0]}")
print(f"factor_exposures for asset 2 {factor_exposure_a[1]}")

## Quiz: Variance of stock 1

Calculate the variance of stock 1.  
$\textrm{Var}(f_{1}) = \beta_{1,1}^2 \textrm{Var}(f_{1}) + \beta_{1,2}^2 \textrm{Var}(f_{2}) + 2\beta_{1,1}\beta_{1,2}\textrm{Cov}(f_{1},f_{2}) + \textrm{Var}(s_{1})$

In [ ]:
factor_exposure_1_1 = factor_exposure_a[0][0]
factor_exposure_1_2 = factor_exposure_a[0][1]
common_return_1 = factor_exposure_1_1 * factor_return_1 + factor_exposure_1_2 * factor_return_2
specific_return_1 = asset_return_1 - common_return_1

In [ ]:
covm_f1_f2 = np.cov(factor_return_1,factor_return_2,ddof=1) #this calculates a covariance matrix
# TODO: get the variance of each factor, and covariances from the covariance matrix covm_f1_f2
var_f1 = #...
var_f2 = #...
cov_f1_f2 = #...

# TODO: calculate the specific variance.  
var_s_1 = # ...

# TODO: calculate the variance of asset 1 in terms of the factors and specific variance
var_asset_1 = # ...
print(f"variance of asset 1: {var_asset_1:.8f}")

## Quiz 2: Variance of stock 2
Calculate the variance of stock 2.  
$\textrm{Var}(f_{2}) = \beta_{2,1}^2 \textrm{Var}(f_{1}) + \beta_{2,2}^2 \textrm{Var}(f_{2}) + 2\beta_{2,1}\beta_{2,2}\textrm{Cov}(f_{1},f_{2}) + \textrm{Var}(s_{2})$

In [ ]:
factor_exposure_2_1 = factor_exposure_a[1][0]
factor_exposure_2_2 = factor_exposure_a[1][1]
common_return_2 = factor_exposure_2_1 * factor_return_1 + factor_exposure_2_2 * factor_return_2
specific_return_2 = asset_return_2 - common_return_2

In [ ]:
# Notice we already calculated the variance and covariances of the factors

# TODO: calculate the specific variance of asset 2
var_s_2 = # ...

# TODO: calcualte the variance of asset 2 in terms of the factors and specific variance
var_asset_2 = # ...
            
print(f"variance of asset 2: {var_asset_2:.8f}")

## Quiz 3: Covariance of stocks 1 and 2
Calculate the covariance of stock 1 and 2.  
$\textrm{Cov}(f_{1},f_{2}) = \beta_{1,1}\beta_{2,1}\textrm{Var}(f_{1}) + \beta_{1,1}\beta_{2,2}\textrm{Cov}(f_{1},f_{2}) + \beta_{1,2}\beta_{2,1}\textrm{Cov}(f_{1},f_{2}) + \beta_{1,2}\beta_{2,2}\textrm{Var}(f_{2})$

In [ ]:
# TODO: calculate the covariance of assets 1 and 2 in terms of the factors
cov_asset_1_2 = # ...
print(f"covariance of assets 1 and 2: {cov_asset_1_2:.8f}")

## Quiz 4: Do it with Matrices!

Create matrices $\mathbf{F}$, $\mathbf{B}$ and $\mathbf{S}$, where  
$\mathbf{F}= \begin{pmatrix}
\textrm{Var}(f_1) & \textrm{Cov}(f_1,f_2) \\ 
\textrm{Cov}(f_2,f_1) & \textrm{Var}(f_2) 
\end{pmatrix}$
is the covariance matrix of factors,  

$\mathbf{B} = \begin{pmatrix}
\beta_{1,1}, \beta_{1,2}\\ 
\beta_{2,1}, \beta_{2,2}
\end{pmatrix}$ 
is the matrix of factor exposures, and  

$\mathbf{S} = \begin{pmatrix}
\textrm{Var}(s_i) & 0\\ 
0 & \textrm{Var}(s_j)
\end{pmatrix}$
is the matrix of specific variances.  

Then calculate $\mathbf{BFB}^T$ 

In [ ]:
# TODO: covariance matrix of factors
F = # ...
F

In [ ]:
# TODO: matrix of factor exposures
B = # ...
B

#### Hint: for diagonal matrices
You can try using [numpy.diag](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.diag.html)

In [ ]:
# TODO: matrix of specific variances
S = # ...
S

In [ ]:
# TODO: covariance matrix of assets
covm_assets = # ...
print(f"covariance matrix of assets 1 and 2 \n{covm_assets}")

## Solution
[Solution notebook](covariance_matrix_assets_solution.ipynb)